In [125]:
import requests
from collections import Counter

In [99]:
api_url = 'https://catalog.archives.gov/api/v1/'
# query_url = ''.join([api_url, '?resultTypes=item,fileUnit', '&cursorMark=*'])
query_url = ''.join([api_url, '?resultFields=description.fileUnit.topicalSubjectArray', '&cursorMark=*'])
# query_url = ''.join([api_url, '?resultTypes=fileUnit', '&cursorMark=*'])
query_url

'https://catalog.archives.gov/api/v1/?resultFields=description.fileUnit.topicalSubjectArray&cursorMark=*'

In [100]:
r = requests.get(query_url)
response = r.json()

In [101]:
response.keys()

dict_keys(['opaResponse'])

In [102]:
response['opaResponse'].keys()

dict_keys(['@status', 'time', 'request', 'results'])

In [103]:
# only use this for errors
# for key in response['opaResponse'].keys():
#     print('{}: {}'.format(key, response['opaResponse'][key]))

In [104]:
for key in ['@status', 'time', 'request']:
    print('{}: {}'.format(key, response['opaResponse'][key]))

@status: 200
time: 2019-11-09T23:05:12.556Z
request: {'format': 'json', 'pretty': True, 'action': 'search', 'resultFields': 'description.fileUnit.topicalSubjectArray', 'cursorMark': '*'}


In [105]:
response['opaResponse']['results'].keys()

dict_keys(['queryTime', 'total', 'offset', 'nextCursorMark', 'rows', 'result'])

In [106]:
response['opaResponse']['results']['nextCursorMark']

'AoMIP4AAAHCwy4up7gI3b2JqLTEyNDIxODUxNS0xMzQ3Mzc2Mzc=-10'

In [107]:
response['opaResponse']['results']['total']

121043289

In [108]:
response['opaResponse']['results']['rows']

10

In [109]:
len(response['opaResponse']['results']['result'])

10

In [110]:
response['opaResponse']['results']['result'][0].keys()
# response['opaResponse']['results']['result'][0]

dict_keys([])

In [127]:
response['opaResponse']['results']['result'][2] #['description']['fileUnit']['topicalSubjectArray']['topicalSubject'][0]['termName']

{'description': {'fileUnit': {'topicalSubjectArray': {'topicalSubject': [{'naId': '138671671',
      'termName': 'Air Medal (U.S.)',
      'recordHistory': {'approved': {'modification': {'dateTime': '2019-06-07T15:55:08'}},
       'changed': {'modification': {'dateTime': '2019-06-07T15:54:30'}},
       'created': {'dateTime': '2019-06-07T15:53:44'}}},
     {'naId': '138671671',
      'termName': 'Air Medal (U.S.)',
      'recordHistory': {'approved': {'modification': {'dateTime': '2019-06-07T15:55:08'}},
       'changed': {'modification': {'dateTime': '2019-06-07T15:54:30'}},
       'created': {'dateTime': '2019-06-07T15:53:44'}}}]}}}}

In [128]:
topix = Counter()

for result in response['opaResponse']['results']['result']:
    if result:
        fields = result['description']['fileUnit']['topicalSubjectArray']['topicalSubject']
        if type(fields) == dict:
            topic = result['description']['fileUnit']['topicalSubjectArray']['topicalSubject']['termName']
            topix[topic] += 1
        elif type(fields) == list:
            for tinier_result in result['description']['fileUnit']['topicalSubjectArray']['topicalSubject']:
                topix[tinier_result['termName']] += 1
topix

Counter({'Distinguished Flying Cross (Medal)': 1, 'Air Medal (U.S.)': 2})

In [130]:
topics = Counter()

api_url = 'https://catalog.archives.gov/api/v1/'
query_url = ''.join([api_url, '?resultFields=description.fileUnit.topicalSubjectArray', '&cursorMark=*'])

r = requests.get(query_url)
response = r.json()

while r.ok:
    for result in response['opaResponse']['results']['result']:
        if result:
            fields = result['description']['fileUnit']['topicalSubjectArray']['topicalSubject']
            if type(fields) == dict:
                topic = result['description']['fileUnit']['topicalSubjectArray']['topicalSubject']['termName']
                topics[topic] += 1
            elif type(fields) == list:
                for tinier_result in result['description']['fileUnit']['topicalSubjectArray']['topicalSubject']:
                    topics[tinier_result['termName']] += 1

    next_cursor = response['opaResponse']['results']['nextCursorMark']
    query_url = ''.join([api_url, '?resultFields=description.fileUnit.topicalSubjectArray', '&cursorMark={}'.format(next_cursor)])
    r = requests.get(query_url)
    response = r.json()

In [131]:
topics

Counter({'Distinguished Flying Cross (Medal)': 2,
         'Air Medal (U.S.)': 2,
         'Court martial and courts of inquiry': 171,
         'Naturalization': 1,
         'Fraud': 1,
         'Citizenship': 1,
         'Judges': 1,
         'Federal judges': 1,
         'Diplomatic and consular service': 1,
         'Cabinet officers': 1,
         'Land use': 1,
         'Cities and towns': 1})

## ['description']['fileUnit']

In [31]:
response['opaResponse']['results']['result'][0]['description']['fileUnit']

{'naId': '124218515',
 'accessRestriction': {'status': {'naId': '10031403',
   'termName': 'Unrestricted'}},
 'dataControlGroup': {'groupCd': 'REBO',
  'groupId': 'ou=NRAAB,ou=groups',
  'groupName': "NARA's Northeast Region (Boston)"},
 'generalRecordsTypeArray': {'generalRecordsType': {'naId': '10035676',
   'termName': 'Textual Records'}},
 'isAv': 'false',
 'isUnderEdit': 'false',
 'physicalOccurrenceArray': {'fileUnitPhysicalOccurrence': {'copyStatus': {'naId': '10031434',
    'termName': 'Preservation-Reproduction-Reference'},
   'locationArray': {'location': {'facility': {'naId': '10048519',
      'termName': "NARA's Northeast Region (Boston, MA)"}}},
   'mediaOccurrenceArray': {'mediaOccurrence': {'containerId': 'Box 7',
     'generalMediaTypeArray': {'generalMediaType': {'naId': '12000002',
       'termName': 'Bound Volumes'}},
     'specificMediaType': {'naId': '10048756', 'termName': 'Paper'}}},
   'referenceUnitArray': {'referenceUnit': {'mailCode': 'RE-BO',
     'name': 'N

In [32]:
response['opaResponse']['results']['result'][0]['description']['fileUnit']['accessRestriction']['status']['termName']

'Unrestricted'

In [33]:
response['opaResponse']['results']['result'][0]['description']['fileUnit']['generalRecordsTypeArray']['generalRecordsType']['termName']

'Textual Records'

In [34]:
response['opaResponse']['results']['result'][0]['description']['fileUnit']['physicalOccurrenceArray']['fileUnitPhysicalOccurrence']['mediaOccurrenceArray']['mediaOccurrence']['generalMediaTypeArray']['generalMediaType']['termName']

'Bound Volumes'

In [43]:
response['opaResponse']['results']['result'][0]['description']['fileUnit'].keys()

dict_keys(['naId', 'accessRestriction', 'dataControlGroup', 'generalRecordsTypeArray', 'isAv', 'isUnderEdit', 'physicalOccurrenceArray', 'recordHistory', 'title', 'useRestriction', 'itemCount', 'itemAvCount', 'parentSeries'])

In [35]:
response['opaResponse']['results']['result'][0]['description']['fileUnit']['topicalSubjectArray']['topicalSubject']['termName']

KeyError: 'topicalSubjectArray'

In [41]:
response['opaResponse']['results']['result'][0]['objects']['object'][0]

{'@id': '134737635',
 '@objectSortNum': '476',
 'technicalMetadata': {'size': '512024494',
  'mime': 'application/pdf',
  'createDate': '2019-04-19T15:25:57Z',
  'modifyDate': '2019-04-19T15:25:57Z',
  'Page-Count': '475'},
 'file': {'@mime': 'application/pdf',
  '@name': '124218515.pdf',
  '@path': '/lz/boston/rg-036/6046816/124218515/124218515.pdf',
  '@type': 'primary',
  '@url': 'https://catalog.archives.gov/catalogmedia/lz/boston/rg-036/6046816/124218515/124218515.pdf',
  '@extractedTextFile': 'https://catalog.archives.gov/catalogmedia/live/boston/rg-036/6046816/124218515/124218515.pdf/opa-renditions/extracted-text/124218515.pdf.txt'},
 'thumbnail': {'@mime': 'image/jpeg',
  '@path': 'opa-renditions/thumbnails/124218515.pdf-thumb.jpg',
  '@url': 'https://catalog.archives.gov/catalogmedia/live/boston/rg-036/6046816/124218515/124218515.pdf/opa-renditions/thumbnails/124218515.pdf-thumb.jpg'}}

In [44]:
for result in response['opaResponse']['results']['result']:
    if 'topicalSubjectArray' in result['description']['fileUnit'].keys():
        print(True)
    else:
        print(False)

False
True
True
False


KeyError: 'fileUnit'